In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [17]:
data=pd.read_csv('data_without_scaling')

In [18]:
data.head()

,Unnamed: 0,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,1,0,61.0,0,0,1,3,0,202.21,28.1,2,1
2,2,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,3,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,4,0,79.0,1,0,1,3,0,174.12,24.0,2,1


In [19]:
data.drop('Unnamed: 0',axis=1,inplace=True)

Random Forest Classifier

In [20]:
#independent feature
Y=data['stroke']

In [21]:
data.drop('stroke',axis=1,inplace=True)

In [22]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,67.0,0,1,1,2,1,228.69,36.6,1
1,0,61.0,0,0,1,3,0,202.21,28.1,2
2,1,80.0,0,1,1,2,0,105.92,32.5,2
3,0,49.0,0,0,1,2,1,171.23,34.4,3
4,0,79.0,1,0,1,3,0,174.12,24.0,2


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.33, random_state=42)

In [24]:
X_train.shape

(3423, 10)

In [25]:
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import optuna

In [31]:
#hyperparameters for optuna
def objective(trial) :
    criterions = trial.suggest_categorical('criterion', ["gini", "entropy"])
    max_depths = trial.suggest_int('max_depth', 100,1000,10)
    n_estimators = trial.suggest_int('n_estimators', 100,1000,10)
    min_samples_split=trial.suggest_int('min_samples_split',2,10,1)
    min_samples_leaf=trial.suggest_int('min_samples_leaf',1,10,1)
    
    clf=RandomForestClassifier(n_estimators=n_estimators,
                                 criterion=criterions,
                                 max_depth=max_depths,
                                 min_samples_leaf=min_samples_leaf,
                                 min_samples_split=min_samples_split,            
                                 n_jobs=-1)
    score = cross_val_score(clf, X_train, y_train, scoring="accuracy",cv=3).mean()
    return score

In [32]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)
trial = study.best_trial

[I 2021-06-02 20:05:30,957] A new study created in memory with name: no-name-32c68597-1fd6-4d1b-a9d4-86b5b354a0b1
[I 2021-06-02 20:05:31,540] Trial 0 finished with value: 0.9553023663453112 and parameters: {'criterion': 'gini', 'max_depth': 610, 'n_estimators': 100, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9553023663453112.
[I 2021-06-02 20:05:34,861] Trial 1 finished with value: 0.9553023663453112 and parameters: {'criterion': 'gini', 'max_depth': 120, 'n_estimators': 800, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.9553023663453112.
[I 2021-06-02 20:05:38,585] Trial 2 finished with value: 0.9553023663453112 and parameters: {'criterion': 'entropy', 'max_depth': 930, 'n_estimators': 820, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9553023663453112.
[I 2021-06-02 20:05:40,823] Trial 3 finished with value: 0.9550102249488752 and parameters: {'criterion': 'gini', 'max_depth': 880, 'n_e

In [33]:
study.best_params

{'criterion': 'gini',
 'max_depth': 610,
 'n_estimators': 100,
 'min_samples_split': 9,
 'min_samples_leaf': 2}

In [34]:
rf=RandomForestClassifier(n_estimators=100,max_depth=610,criterion='gini',min_samples_split=9,min_samples_leaf=2)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=610, min_samples_leaf=2, min_samples_split=9)

In [36]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
y_pred=rf.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.943094250148192


XGBoost


In [38]:
import xgboost

In [67]:
classifier=xgboost.XGBClassifier()
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)
print(accuracy_score(y_test,y_pred))

[20:45:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9389448725548311


In [48]:
score

array([0.95267309, 0.94390885, 0.94303243])

In [49]:
score.mean()

0.946538124452235

In [50]:
#hyperparameter optimisation
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [51]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [53]:
random_search.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    6.9s finished
/home/moglix/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:36:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=No

In [57]:

random_search.best_params_

{'min_child_weight': 7,
 'max_depth': 4,
 'learning_rate': 0.05,
 'gamma': 0.2,
 'colsample_bytree': 0.4}

In [59]:
classifier=xgboost.XGBClassifier(min_child_weight=7,max_depth=4,learning_rate=0.05,gamma=0.2,colsample_bytree=0.4)

In [64]:
classifier.fit(X_train,y_train)

[20:44:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
y_pred=classifier.predict(X_test)

In [66]:
print(accuracy_score(y_test,y_pred))

0.943094250148192


Observation :

1. Due to less records after hyperparameter tuning small increase in accuracy is seen.

2. Both models are performing good.